In [122]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense, GRU
import numpy as np
import pandas as pd

import enigma_model as em

In [41]:
# lines = open('fra.txt', encoding='utf-8').read().split('\n')

In [42]:
# nb_samples = 1<<16
# plain, cipher = em.generate_data(nb_samples)
# enigma_data = pd.DataFrame({'PLAIN': plain, 'CIPHER': cipher})
# enigma_data.head()

In [43]:
# enigma_data.to_csv('./enigma_data_15.csv', index=False)

In [123]:
enigma_data = pd.read_csv('./enigma_data_15.csv')
len(enigma_data)

test = enigma_data.sample(n=16384, replace=False, random_state=42)
enigma_data = enigma_data.drop(test.index)
print(len(test))
print(len(enigma_data))
nb_samples = 16384

16384
16384


In [124]:
def divide_chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

df = enigma_data.copy()
df

enigma_data_partitions = {
    'ID': [], 
    'PLAIN': [], 
    'CIPHER': []
}

n = 10

for index, row in df.iterrows():    
    plain = list(str(row['PLAIN']))
    cipher = list(str(row['CIPHER']))
    
    plain = list(divide_chunks(lst=plain, n=n))
    cipher = list(divide_chunks(lst=cipher, n=n))
    
    for i in range(len(plain)):
        plain_now = "".join(plain[i])
        cipher_now = "".join(cipher[i])
        
        enigma_data_partitions['ID'].append(index)
        enigma_data_partitions['PLAIN'].append(plain_now)
        enigma_data_partitions['CIPHER'].append(cipher_now)
    
enigma_data_partitions = pd.DataFrame(enigma_data_partitions)
enigma_data_partitions.head(10)

,ID,PLAIN,CIPHER
0,1,NOTICEKIDS,AMJAEZZGJN
1,1,UPPORTDECI,RTZESCHLEE
2,1,DE,LO
3,2,CHOICECHAR,VNBAEZMOQG
4,2,ACTERTECHN,KUCOSCUMJY
5,2,OLOGYSERIO,TKEIZUGLVU
6,2,USLAND,MZJWJE
7,5,OTHERLISTE,PBAKMUXWYQ
8,5,NMANAGEMEN,JHDQKEUCCY
9,5,TAHEADCASE,OBFNULDZHW


In [57]:
# enigma_test_run = enigma_data_partitions.query('ID <= 4000')

# len(enigma_test_run)

In [125]:
eng_sent = []
fra_sent = []
eng_chars = set()
fra_chars = set()
# nb_samples = enigma_data.shape[0]

# Process english and french sentences
for index, row in enigma_data_partitions.iterrows():
#     eng_line = str(lines[line]).split('\t')[0]
    eng_line = str(row['CIPHER'])
    
    # Append '\t' for start of the sentence and '\n' to signify end of the sentence
#     fra_line = '\t' + str(lines[line]).split('\t')[1] + '\n'
    fra_line = f"\t{str(row['PLAIN'])}\n"
    
    eng_sent.append(eng_line)
    fra_sent.append(fra_line)
    
    for ch in eng_line:
        if (ch not in eng_chars):
            eng_chars.add(ch)
            
    for ch in fra_line:
        if (ch not in fra_chars):
            fra_chars.add(ch)

In [126]:
print(len(eng_sent), len(eng_sent))

print(eng_sent[0], fra_sent[0])

49701 49701
AMJAEZZGJN 	NOTICEKIDS



In [49]:
enigma_test_run.head()

NameError: name 'enigma_test_run' is not defined

In [127]:
fra_chars = sorted(list(fra_chars))
eng_chars = sorted(list(eng_chars))

In [128]:
print(eng_sent[0])
print(fra_sent[0].strip())
print(eng_chars)
print(fra_chars)

AMJAEZZGJN
NOTICEKIDS
['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
['\t', '\n', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']


In [129]:
# dictionary to index each english character - key is index and value is english character
eng_index_to_char_dict = {}

# dictionary to get english character given its index - key is english character and value is index
eng_char_to_index_dict = {}

for k, v in enumerate(eng_chars):
    eng_index_to_char_dict[k] = v
    eng_char_to_index_dict[v] = k

In [130]:
# dictionary to index each french character - key is index and value is french character
fra_index_to_char_dict = {}

# dictionary to get french character given its index - key is french character and value is index
fra_char_to_index_dict = {}
for k, v in enumerate(fra_chars):
    fra_index_to_char_dict[k] = v
    fra_char_to_index_dict[v] = k

In [131]:
max_len_eng_sent = max([len(line) for line in eng_sent])
max_len_fra_sent = max([len(line) for line in fra_sent])

In [132]:
print(max_len_eng_sent)
print(max_len_fra_sent)

print(len(eng_sent))

10
12
49701


In [133]:
nb_samples = 49701
tokenized_eng_sentences = np.zeros(shape = (nb_samples,max_len_eng_sent,len(eng_chars)), dtype='float32')
tokenized_fra_sentences = np.zeros(shape = (nb_samples,max_len_fra_sent,len(fra_chars)), dtype='float32')
target_data = np.zeros((nb_samples, max_len_fra_sent, len(fra_chars)),dtype='float32')

In [134]:
# Vectorize the english and french sentences

nb_samples = 49701
for i in range(nb_samples):
    for k,ch in enumerate(eng_sent[i]):
        tokenized_eng_sentences[i,k,eng_char_to_index_dict[ch]] = 1
        
    for k,ch in enumerate(fra_sent[i]):
        tokenized_fra_sentences[i,k,fra_char_to_index_dict[ch]] = 1

        # decoder_target_data will be ahead by one timestep and will not include the start character.
        if k > 0:
            target_data[i,k-1,fra_char_to_index_dict[ch]] = 1

In [135]:
# Encoder model

encoder_input = Input(shape=(None,len(eng_chars)))
encoder_LSTM = LSTM(256,return_state = True)
encoder_outputs, encoder_h, encoder_c = encoder_LSTM (encoder_input)
encoder_states = [encoder_h, encoder_c]


In [136]:
# Decoder model

decoder_input = Input(shape=(None,len(fra_chars)))
decoder_LSTM = LSTM(256,return_sequences=True, return_state = True)
decoder_out, _ , _ = decoder_LSTM(decoder_input, initial_state=encoder_states)
decoder_dense = Dense(len(fra_chars),activation='softmax')
decoder_out = decoder_dense(decoder_out)


In [137]:
from keras.optimizers import Adam

model = Model(inputs=[encoder_input, decoder_input],outputs=[decoder_out])

# Run training
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy')
model.fit(x=[tokenized_eng_sentences,tokenized_fra_sentences], 
          y=target_data,
          batch_size=256,
          epochs=100,
          validation_split=0.3)

Train on 34790 samples, validate on 14911 samples
Epoch 1/100
34790/34790 [==============================] - 18s 530us/step - loss: 2.1885 - val_loss: 2.2747
Epoch 2/100
34790/34790 [==============================] - 15s 443us/step - loss: 2.0114 - val_loss: 1.9876
Epoch 3/100
34790/34790 [==============================] - 16s 454us/step - loss: 1.8750 - val_loss: 1.8030
Epoch 4/100
34790/34790 [==============================] - 16s 451us/step - loss: 1.7575 - val_loss: 1.7145
Epoch 5/100
34790/34790 [==============================] - 17s 500us/step - loss: 1.7866 - val_loss: 1.6619
Epoch 6/100
34790/34790 [==============================] - 17s 496us/step - loss: 1.6792 - val_loss: 1.6082
Epoch 7/100
34790/34790 [==============================] - 18s 518us/step - loss: 1.5868 - val_loss: 1.5746
Epoch 8/100
34790/34790 [==============================] - 18s 521us/step - loss: 1.4960 - val_loss: 1.4611
Epoch 9/100
34790/34790 [==============================] - 16s 467us/step - loss: 1.43

Epoch 76/100
34790/34790 [==============================] - 17s 503us/step - loss: 0.0060 - val_loss: 0.3175
Epoch 77/100
34790/34790 [==============================] - 18s 522us/step - loss: 0.0054 - val_loss: 0.3208
Epoch 78/100
34790/34790 [==============================] - 18s 526us/step - loss: 0.0050 - val_loss: 0.3259
Epoch 79/100
34790/34790 [==============================] - 18s 526us/step - loss: 0.0047 - val_loss: 0.3315
Epoch 80/100
34790/34790 [==============================] - 19s 536us/step - loss: 0.0046 - val_loss: 0.3330
Epoch 81/100
34790/34790 [==============================] - 18s 522us/step - loss: 0.0045 - val_loss: 0.3376
Epoch 82/100
34790/34790 [==============================] - 19s 537us/step - loss: 0.0044 - val_loss: 0.3406
Epoch 83/100
34790/34790 [==============================] - 18s 530us/step - loss: 0.0042 - val_loss: 0.3424
Epoch 84/100
34790/34790 [==============================] - 18s 519us/step - loss: 0.0041 - val_loss: 0.3432
Epoch 85/100
34790/

In [138]:
# Inference models for testing

# Encoder inference model
encoder_model_inf = Model(encoder_input, encoder_states)

# Decoder inference model
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_input_states = [decoder_state_input_h, decoder_state_input_c]

decoder_out, decoder_h, decoder_c = decoder_LSTM(decoder_input, 
                                                 initial_state=decoder_input_states)

decoder_states = [decoder_h , decoder_c]

decoder_out = decoder_dense(decoder_out)

decoder_model_inf = Model(inputs=[decoder_input] + decoder_input_states,
                          outputs=[decoder_out] + decoder_states )

In [139]:
def decode_seq(inp_seq):
    
    # Initial states value is coming from the encoder 
    states_val = encoder_model_inf.predict(inp_seq)
    
    target_seq = np.zeros((1, 1, len(fra_chars)))
    target_seq[0, 0, fra_char_to_index_dict['\t']] = 1
    
    translated_sent = ''
    stop_condition = False
    
    while not stop_condition:
        
        decoder_out, decoder_h, decoder_c = decoder_model_inf.predict(x=[target_seq] + states_val)
        
        max_val_index = np.argmax(decoder_out[0,-1,:])
        sampled_fra_char = fra_index_to_char_dict[max_val_index]
        translated_sent += sampled_fra_char
        
        if ( (sampled_fra_char == '\n') or (len(translated_sent) > max_len_fra_sent)) :
            stop_condition = True
        
        target_seq = np.zeros((1, 1, len(fra_chars)))
        target_seq[0, 0, max_val_index] = 1
        
        states_val = [decoder_h, decoder_c]
        
    return translated_sent



In [140]:
def divide_chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

df = test.copy()
df

enigma_test_partitions = {
    'ID': [], 
    'PLAIN': [], 
    'CIPHER': []
}

n = 10

for index, row in df.iterrows():    
    plain = list(str(row['PLAIN']))
    cipher = list(str(row['CIPHER']))
    
    plain = list(divide_chunks(lst=plain, n=n))
    cipher = list(divide_chunks(lst=cipher, n=n))
    
    for i in range(len(plain)):
        plain_now = "".join(plain[i])
        cipher_now = "".join(cipher[i])
        
        enigma_test_partitions['ID'].append(index)
        enigma_test_partitions['PLAIN'].append(plain_now)
        enigma_test_partitions['CIPHER'].append(cipher_now)
    
enigma_test_partitions = pd.DataFrame(enigma_test_partitions)
enigma_test_partitions.head(10)
len(enigma_test_partitions)

49645

In [141]:
nb_samples_test = 49645
eng_sent_test = []
fra_sent_test = []
eng_chars_test = set()
fra_chars_test = set()
# nb_samples = enigma_data.shape[0]

# Process english and french sentences
for index, row in enigma_test_partitions.iterrows():
#     eng_line = str(lines[line]).split('\t')[0]
    eng_line = str(row['CIPHER'])
    
    # Append '\t' for start of the sentence and '\n' to signify end of the sentence
#     fra_line = '\t' + str(lines[line]).split('\t')[1] + '\n'
    fra_line = f"\t{str(row['PLAIN'])}\n"
    
    eng_sent_test.append(eng_line)
    fra_sent_test.append(fra_line)
    
    for ch in eng_line:
        if (ch not in eng_chars_test):
            eng_chars_test.add(ch)
            
    for ch in fra_line:
        if (ch not in fra_chars_test):
            fra_chars_test.add(ch)
fra_chars_test = sorted(list(fra_chars_test))
eng_chars_test = sorted(list(eng_chars_test))

# dictionary to index each english character - key is index and value is english character
eng_index_to_char_dict_test = {}

# dictionary to get english character given its index - key is english character and value is index
eng_char_to_index_dict_test = {}

for k, v in enumerate(eng_chars_test):
    eng_index_to_char_dict_test[k] = v
    eng_char_to_index_dict_test[v] = k

max_len_eng_sent_test = max([len(line) for line in eng_sent_test])
max_len_fra_sent_test = max([len(line) for line in fra_sent_test])



tokenized_eng_sentences_test = np.zeros(shape = (nb_samples_test,max_len_eng_sent_test,len(eng_chars_test)), dtype='float32')
tokenized_fra_sentences_test = np.zeros(shape = (nb_samples_test,max_len_fra_sent_test,len(fra_chars_test)), dtype='float32')
target_data_test = np.zeros((nb_samples_test, max_len_fra_sent_test, len(fra_chars_test)),dtype='float32')


# Vectorize the english and french sentences

for i in range(nb_samples_test):
    for k,ch in enumerate(eng_sent_test[i]):
        tokenized_eng_sentences_test[i,k,eng_char_to_index_dict_test[ch]] = 1
        
#     for k,ch in enumerate(fra_sent_test[i]):
#         tokenized_fra_sentences_test[i,k,fra_char_to_index_dict_test[ch]] = 1

#         # decoder_target_data will be ahead by one timestep and will not include the start character.
#         if k > 0:
#             target_data_test[i,k-1,fra_char_to_index_dict_test[ch]] = 1

In [22]:


for seq_index in range(20):
    inp_seq = tokenized_eng_sentences_test[seq_index:seq_index+1]
    translated_sent = decode_seq(inp_seq)
    print('-' * 100)
    print('Input sentence:', eng_sent_test[seq_index])
    print('Decoded sentence:', translated_sent.strip())
    print('decode Org sentence:', fra_sent_test[seq_index])



----------------------------------------------------------------------------------------------------
Input sentence: JEJLCWSG
Decoded sentence: FATHERDI
decode Org sentence: 	FATHERDI

----------------------------------------------------------------------------------------------------
Input sentence: OFBNBZBD
Decoded sentence: SCOVERPR
decode Org sentence: 	SCOVERPR

----------------------------------------------------------------------------------------------------
Input sentence: FMCONUCB
Decoded sentence: ICEFIRHT
decode Org sentence: 	ICEFIRST

----------------------------------------------------------------------------------------------------
Input sentence: PJDKYWTT
Decoded sentence: FACTREAD
decode Org sentence: 	FACTREAD

----------------------------------------------------------------------------------------------------
Input sentence: IASHKLVJ
Decoded sentence: WELLFUTU
decode Org sentence: 	WELLFUTU

---------------------------------------------------------------------------

In [142]:
# test.head()
nb_samples_test = 49645


predicted_cipher = []
# actual_cipher = []

# cipher = list(test['CIPHER'])

for seq_index in range(nb_samples_test):
    inp_seq = tokenized_eng_sentences_test[seq_index:seq_index+1]
    translated_sent = decode_seq(inp_seq)
    predicted_cipher.append(translated_sent)
#     actual_cipher.append(fra_sent_test[seq_index])
    
#     print('-' * 100)
#     print('Input sentence:', eng_sent_test[seq_index])
#     print('Decoded sentence:', translated_sent.strip())
#     print('Decoded original:', cipher[seq_index])

   
    
# def str_score(str_a, str_b) :
# #     if len(str_a) != len(str_b):
# #         return 0

#     n_correct = 0

#     for a, b in zip(str_a, str_b):
#         n_correct += int(a == b)
#     # print(f" n_correct {n_correct}")
#     # print(f" len  {n_correct}")

#     return n_correct / len(str_a)

    
# def score(predicted_plain, correct_plain):
#     correct = 0

#     for p, c in zip(predicted_plain, correct_plain):
#         print(p,c)
#         exit()
#         if str_score(p, c) > 0.8:
#             correct += 1
#     print(f" correct {correct}")
#     print(f" len correct_plain {len(correct_plain)}")

#     return correct / len(correct_plain)


# print(predicted_cipher)
# print(actual_cipher)


# print(score(predicted_cipher, actual_cipher ))



In [147]:

# predicted_cipher = pd.DataFrame(predicted_cipher)

# predicted_cipher.save_csv('./predicted_english_seperated')

print(enigma_test_partitions.head())

predicted_cipher[0:10]


     ID       PLAIN      CIPHER
0  1588  FATHERDISC  JEJLCWSGOF
1  1588  OVERPRICEF  BNBZBDFMCO
2  1588  IRSTFACTRE  NUCBPJDKYW
3  1588          AD          TT
4  2153  WELLFUTURE  IASHKLVJZQ


['FATHERDISCOVERPRICEFIRSTACHEREAD',
 'WELLFUTUREMANAGEMENT',
 'OVEREXISTCOLDCAPITAL',
 'FRIENDMILITARYEATLEAST',
 'FILLMEANHEALTHRELWIRE',
 'RETITTOANDTHINGUREFFORTFIGHTWIAST',
 'THROUGHOUTBILLTARIF',
 'STYLEPEOPLEFORCEPUTMODERN',
 'ENOUGSUBLPARTYPARENTENOUGH',
 'BAXPACTFACEALONEWITHOUT']

In [32]:
predicted_cipher = predicted_cipher.iloc[:, 0].tolist()

In [148]:
enigma_predicted = enigma_test_partitions.copy()
enigma_predicted['DECRYPTED'] = [x.strip() for x in predicted_cipher]
enigma_predicted.head(20)


enigma_predicted.to_csv('./enigma_predicted_n10.csv')

ValueError: Length of values does not match length of index

In [145]:
# foo = enigma_predicted.groupby(['ID'], as_index=False, sort=False) \
#     .agg(
#     {
#         'PLAIN': ''.join,
#         'CIPHER': ''.join,
#         'DECPRYPTED': ''.join
#     })

# foo = enigma_predicted[['ID', 'DECRYPTED']] \
#     .groupby(['ID'], as_index=False, sort=False) \
#     .agg(''.join)

foo = enigma_predicted.groupby(['ID'], as_index=False, sort=False) \
    .agg(''.join)

foo.head()

foo.to_csv('./joined_predicted_cipher_n10.csv')

In [ ]:
# df.agg({'A' : ['sum', 'min'], 'B' : ['min', 'max']})
# df.groupby(['vid', 'sente'], as_index=False, sort=False).agg(' '.join)

In [146]:
 
def str_score(str_a, str_b) :
    if len(str_a) != len(str_b):
        return 0

    n_correct = 0

    for a, b in zip(str_a, str_b):
        n_correct += int(a == b)
    # print(f" n_correct {n_correct}")
    # print(f" len  {n_correct}")

    return n_correct / len(str_a)

    
def score(predicted_plain, correct_plain):
    correct = 0

    for p, c in zip(predicted_plain, correct_plain):
#         print(p,c)
#         exit()
        if str_score(p, c) > 0.8:
            correct += 1
    print(f" correct {correct}")
    print(f" len correct_plain {len(correct_plain)}")

    return correct / len(correct_plain)


# print(predicted_cipher)
# print(actual_cipher)

predicted_cipher = list(foo['DECRYPTED'])
actual_cipher = list(foo['PLAIN'])
print(score(predicted_cipher, actual_cipher ))




 correct 12145
 len correct_plain 16384
0.74127197265625
